In [1]:
from MixedAutoencoder import Mixer, MixedAutoencoder
import MixedAutoencoder
from DataCleaning import *
import DataCleaning
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
seed = 42
MixedAutoencoder.setRandom(seed)
DataCleaning.setRandom(seed)
base_path = "."
mixer = Mixer(base_path)

In [2]:

num_sets = 4
latent_dim = 7
model_shape = []
base_path = "."
label = f'demo_{num_sets}_{latent_dim}_[{"_".join([str(s) for s in model_shape])}]'
demo_size = 15
demo_shape = []


In [3]:
base_key_list = [f'set{str(i)}' for i in range(1, num_sets+1)]
demo_key_list = ['demo']
data = pd.read_csv(f'{base_path}/data/16PF/data.csv', sep="\t")
data = clean_data(data)
data = data.sample(frac=1)
demo_column_keys = {demo_key_list[0]: split_n_strat(data, demo_size)}

base_column_keys = split_strat([k for k in list(data) if k not in demo_column_keys[demo_key_list[0]]],num_sets, base_key_list)
split_data = split(data, base_column_keys)
train, test = make_train_test(split_data, 0.8)
input_dims = get_input_dims(train)

In [4]:
model_shapes = {k: model_shape for k in base_key_list}
autoencoder_set = mixer.make_new(model_shapes, latent_dim, input_dims)
autoencoder_set.show_total_binary_accuracy(test)
settings = {
    "training": [["$all", "$all", True, True]]#[[[k1], [k2 for k2 in keys if k1 != k2], True, True] for k1 in keys],
    #"encoder_proximity_training": [["$all", True]],
    #"plot": [True, 3, [0, 1, 2]]
}
autoencoder_set.train_set(train, 25, autoencoder_set.make_train_config(settings = settings), batch_size = 64, verbose=True)
autoencoder_set.show_total_binary_accuracy(test)

Binary accuracy: 0.5083089518449392
epoch 0
442/442 [==============================] - 5s 9ms/step - loss: 1.1875 - val_loss: 1.2687 - accuracy: 0.2728
epoch 1
442/442 [==============================] - 4s 6ms/step - loss: 1.0747 - val_loss: 1.1839 - accuracy: 0.3091
epoch 2
442/442 [==============================] - 4s 5ms/step - loss: 0.9805 - val_loss: 1.1015 - accuracy: 0.3318
epoch 3
442/442 [==============================] - 4s 5ms/step - loss: 0.9296 - val_loss: 1.0554 - accuracy: 0.3491
epoch 4
442/442 [==============================] - 4s 6ms/step - loss: 0.8998 - val_loss: 1.0259 - accuracy: 0.3680
epoch 5
442/442 [==============================] - 4s 6ms/step - loss: 0.8776 - val_loss: 0.9971 - accuracy: 0.3817
epoch 6
442/442 [==============================] - 4s 6ms/step - loss: 0.8622 - val_loss: 0.9801 - accuracy: 0.3898
epoch 7
442/442 [==============================] - 4s 6ms/step - loss: 0.8518 - val_loss: 0.9713 - accuracy: 0.3941
epoch 8
442/442 [===================

In [5]:
demo_split_data = split(data, demo_column_keys)
dtrain, dtest = make_train_test(demo_split_data, 0.8)
demo_input_dims = get_input_dims(dtrain)




demo_shapes = {k: demo_shape for k in demo_key_list}
autoencoder_set = mixer.add_new(autoencoder_set, demo_shapes, demo_input_dims)

settings = {
    "training": [[demo_key_list, "$all", True, False], ["$all", demo_key_list, False, True]]#[[[k1], [k2 for k2 in keys if k1 != k2], True, True] for k1 in keys],
    #"encoder_proximity_training": [["$all", True]],
    #"plot": [True, 3, [0, 1, 2]]
}
autoencoder_set.train_set(train | dtrain, 25, autoencoder_set.make_train_config(settings = settings), batch_size = 64, verbose=True)


epoch 0
442/442 [==============================] - 3s 6ms/step - loss: 1.4392 - val_loss: 1.4735 - accuracy: 0.2871
epoch 1
442/442 [==============================] - 3s 5ms/step - loss: 1.1562 - val_loss: 1.2036 - accuracy: 0.3149
epoch 2
442/442 [==============================] - 3s 5ms/step - loss: 1.0057 - val_loss: 1.0546 - accuracy: 0.3420
epoch 3
442/442 [==============================] - 3s 5ms/step - loss: 0.9315 - val_loss: 0.9792 - accuracy: 0.3654
epoch 4
442/442 [==============================] - 3s 5ms/step - loss: 0.8913 - val_loss: 0.9398 - accuracy: 0.3873
epoch 5
442/442 [==============================] - 3s 5ms/step - loss: 0.8702 - val_loss: 0.9217 - accuracy: 0.3938
epoch 6
442/442 [==============================] - 3s 5ms/step - loss: 0.8597 - val_loss: 0.9143 - accuracy: 0.3927
epoch 7
442/442 [==============================] - 3s 5ms/step - loss: 0.8545 - val_loss: 0.9113 - accuracy: 0.3943
epoch 8
442/442 [==============================] - 3s 4ms/step - loss: 0

In [7]:
autoencoder_set.show_binary_accuracy(demo_key_list, test | dtest)

Encoder binary accuracy: 0.769688130325802
Decoder binary accuracy: 0.8243201229527429


In [6]:

mixer.save_to_label(autoencoder_set, extra = {"columns" : base_column_keys|demo_column_keys}, label = label)

INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/encoder_set1\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/decoder_set1\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/encoder_set2\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/decoder_set2\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/encoder_set3\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/decoder_set3\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/encoder_set4\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/decoder_set4\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/encoder_demo\assets
INFO:tensorflow:Assets written to: ./Models/model_demo_4_7_[]/decoder_demo\assets
